In [27]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date

In [28]:
team_urls = pd.read_csv('Spiele_Links.csv')
idx = 0
print(len(team_urls))
url = team_urls['url'][idx]
response = requests.get(url)

9


In [29]:
#BeautifulSoup HTML-Dokument aus dem Quelltext parsen
soup= BeautifulSoup(response.text, 'html.parser')

In [30]:
#Tabelle aus dem HTML-Dokument extrahieren
table = soup.find('div', {"id":'id-team-matchplan-table'})
#print(table)

In [31]:
team=team_urls['team'][idx]
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    dates = list()
    times = list()
    teams = list()
    kw = list()
    for i in range(1,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        date = rows[i].find('td', class_='column-date') #rows[i].text
        cleanString = re.sub(r'\ \|',',', date.text)
        cleanString = re.sub(r' ', '', cleanString)
        #removedSpace = re.sub(r'\t', '', removedSpace)
        splitted= cleanString.split(',')
        date_split = splitted[1].split('.')
        datum = datetime.date(int(date_split[2])+2000, int(date_split[1]), int(date_split[0]))
        kw.append(datum.isocalendar()[1])
        dates.append(splitted[1])
        times.append(splitted[2].encode('utf-8').decode('utf-8'))
        teams.append(team)
        
df = pd.DataFrame(list(zip(dates,times,teams)),columns=['Datum','Zeit','Team'])
df.insert(len(df.columns), 'KW', kw)
df

,Datum,Zeit,Team,KW
0,29.04.23,16:15,A,17
1,06.05.23,16:15,A,18
2,09.05.23,19:30,A,19
3,13.05.23,16:15,A,19
4,20.05.23,16:15,A,20
5,14.06.23,19:00,A,24
6,17.06.23,16:15,A,24


In [32]:
#Club und Logo auslesen (befindet sich in jeder 3. Zeile ab 3. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    club1 = list()
    club2 = list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        #print(rows[i].text)
        removedSpace = re.sub(r'\n', '', rows[i].text)
        removedSpace = re.sub(r'\t', '', removedSpace)
        removedSpace = re.sub(r':', ';', removedSpace)
        splitted = removedSpace.split(';')
        club1.append(splitted[0])
        club2.append(splitted[1][0:-1])

df.insert(len(df.columns), "Heim", club1)
df.insert(len(df.columns), "Gast", club2)

In [33]:
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    logos_home = list()
    logos_guest= list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        logo = rows[i].find_all('div', class_="club-logo") #rows[i].text
        
        logo_link = re.findall(r'//(\S+)\">', str(logo))
        logos_home.append(logo_link[0])
        logos_guest.append(logo_link[1])
        
#found = re.search('AAA(.+?)ZZZ', string).group(1)

df.insert(len(df.columns), 'Logo Heim', logos_home)
df.insert(len(df.columns), 'Logo Gast', logos_guest)

In [34]:
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast
0,29.04.23,16:15,A,17,SGM TSV Brettach KoBra,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
1,06.05.23,16:15,A,18,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,SGM Neuenstadt Kochertal,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
2,09.05.23,19:30,A,19,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,TSV Talheim,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
3,13.05.23,16:15,A,19,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,Aramäer Heilbronn,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
4,20.05.23,16:15,A,20,Türkspor Neckarsulm,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
5,14.06.23,19:00,A,24,TSV Talheim,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
6,17.06.23,16:15,A,24,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,SGM TSV Brettach KoBra,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...


In [35]:
links = table.find_all('a')
homeLinks = list();
guestLinks = list();
spielLinks = list();
for i in range(2, len(links),6):
    homeLinks.append(links[i].get('href'))
    guestLinks.append(links[i+1].get('href'))
    spielLinks.append(links[i+2].get('href'))
df.insert(len(df.columns), "home_link", homeLinks)
df.insert(len(df.columns), 'guest_link', guestLinks)
df.insert(len(df.columns), 'Spiel', spielLinks)
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast,home_link,guest_link,Spiel
0,29.04.23,16:15,A,17,SGM TSV Brettach KoBra,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-tsv-bre...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/sgm-tsv-brettach...
1,06.05.23,16:15,A,18,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,SGM Neuenstadt Kochertal,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/sgm-neuenst...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...
2,09.05.23,19:30,A,19,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,TSV Talheim,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/tsv-talheim...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...
3,13.05.23,16:15,A,19,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,Aramäer Heilbronn,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/aramaeer-he...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...
4,20.05.23,16:15,A,20,Türkspor Neckarsulm,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/tuerkspor-n...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/tuerkspor-neckar...
5,14.06.23,19:00,A,24,TSV Talheim,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/tsv-talheim...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/tsv-talheim-sgm-...
6,17.06.23,16:15,A,24,SGM SC Abstatt/​Beilstein/​Ilsfeld ABI,SGM TSV Brettach KoBra,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/sgm-tsv-bre...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...


In [36]:
#merge columns to table
if (idx==0):
    df.to_csv('out.csv', index=False, encoding="utf-8")
else:
    df.to_csv('out.csv', mode='a', index=False, header=False, encoding="utf-8")

